# Advent of Code 2022
## Day 19
*<https://adventofcode.com/2022/day/19>*

In [1]:
import IPython
import math
import re
from new_helper import *
from itertools import product, combinations, permutations
from collections import Counter, defaultdict, deque
from string import ascii_lowercase, ascii_uppercase, ascii_letters
from rich import inspect, print, pretty

pretty.install()

In [2]:
DAY = 19
inp = get_aoc_input(DAY, 2022)
part_1 = part_2 = 0

In [3]:
inp = inp.parse_lines()

In [4]:
class Blueprint:
    def __init__(
        self,
        n: int,
        ore_cost_ore: int,
        clay_cost_ore: int,
        obsidian_cost_ore: int,
        obsidian_cost_clay: int,
        geode_cost_ore: int,
        geode_cost_obsidian: int,
    ):
        self.n = n
        self.ore_cost_ore = ore_cost_ore
        self.clay_cost_ore = clay_cost_ore
        self.obsidian_cost_ore = obsidian_cost_ore
        self.obsidian_cost_clay = obsidian_cost_clay
        self.geode_cost_ore = geode_cost_ore
        self.geode_cost_obsidian = geode_cost_obsidian

    def __repr__(self) -> str:
        return (
            f"Blueprint {self.n} = "
            + f"(ORE: {self.ore_cost_ore} ore), "
            + f"(CLAY: {self.clay_cost_ore} ore), "
            + f"(OBSDIAN: {self.obsidian_cost_ore} ore, {self.obsidian_cost_clay} clay), "
            + f"(GEODE: {self.geode_cost_ore} ore, {self.geode_cost_obsidian} obsidian)"
        )

In [5]:
def parse_blueprint(line: str) -> Blueprint:
    n, ore_cost, clay_cost, obsidian_cost_1, obsidian_cost_2, geode_cost_1, geode_cost_2 = ints(line)
    return Blueprint(n, ore_cost, clay_cost, obsidian_cost_1, obsidian_cost_2, geode_cost_1, geode_cost_2)

In [6]:
def eval_blueprint(B: Blueprint, max_time: int) -> int:
    best_geodes = 0
    queue: deque[tuple[int, ...]] = deque([(0, 0, 0, 0, 1, 0, 0, 0, 0)])
    visited = set()

    ore_cap = max(B.ore_cost_ore, B.clay_cost_ore, B.obsidian_cost_ore, B.geode_cost_ore)

    while queue:
        ore, clay, obsidian, geodes, ore_tick, clay_tick, obsidian_tick, geode_tick, tick = queue.popleft()

        if tick == max_time:
            best_geodes = max(best_geodes, geodes)
            continue

        ore_tick = min(ore_tick, ore_cap)
        clay_tick = min(clay_tick, B.obsidian_cost_clay)
        obsidian_tick = min(obsidian_tick, B.geode_cost_obsidian)

        time_left = max_time - tick
        ore = min(ore, time_left * ore_cap - ore_tick * (time_left - 1))
        clay = min(clay, time_left * B.obsidian_cost_clay - clay_tick * (time_left - 1))
        obsidian = min(obsidian, time_left * B.geode_cost_obsidian - obsidian_tick * (time_left - 1))

        state = (ore, clay, obsidian, geodes, ore_tick, clay_tick, obsidian_tick, geode_tick, tick)
        if state in visited:
            continue
        visited.add(state)

        queue.append(
            (
                ore + ore_tick,
                clay + clay_tick,
                obsidian + obsidian_tick,
                geodes + geode_tick,
                ore_tick,
                clay_tick,
                obsidian_tick,
                geode_tick,
                tick + 1,
            )
        )

        if ore >= B.ore_cost_ore:
            queue.append(
                (
                    ore + ore_tick - B.ore_cost_ore,
                    clay + clay_tick,
                    obsidian + obsidian_tick,
                    geodes + geode_tick,
                    ore_tick + 1,
                    clay_tick,
                    obsidian_tick,
                    geode_tick,
                    tick + 1,
                )
            )

        if ore >= B.clay_cost_ore:
            queue.append(
                (
                    ore + ore_tick - B.clay_cost_ore,
                    clay + clay_tick,
                    obsidian + obsidian_tick,
                    geodes + geode_tick,
                    ore_tick,
                    clay_tick + 1,
                    obsidian_tick,
                    geode_tick,
                    tick + 1,
                )
            )

        if ore >= B.obsidian_cost_ore and clay >= B.obsidian_cost_clay:
            queue.append(
                (
                    ore + ore_tick - B.obsidian_cost_ore,
                    clay + clay_tick - B.obsidian_cost_clay,
                    obsidian + obsidian_tick,
                    geodes + geode_tick,
                    ore_tick,
                    clay_tick,
                    obsidian_tick + 1,
                    geode_tick,
                    tick + 1,
                )
            )

        if ore >= B.geode_cost_ore and obsidian >= B.geode_cost_obsidian:
            queue.append(
                (
                    ore + ore_tick - B.geode_cost_ore,
                    clay + clay_tick,
                    obsidian + obsidian_tick - B.geode_cost_obsidian,
                    geodes + geode_tick,
                    ore_tick,
                    clay_tick,
                    obsidian_tick,
                    geode_tick + 1,
                    tick + 1,
                )
            )

    return best_geodes

In [7]:
blueprints = list(map(parse_blueprint, inp))
part_1 = sum(n * eval_blueprint(blueprint, 24) for n, blueprint in enumerate(blueprints, 1))
part_2 = math.prod(eval_blueprint(blueprint, 32) for blueprint in blueprints[:3])

In [8]:
print_part_1(part_1)
print_part_2(part_2)

Part 1: 1418

Part 2: 4114

In [9]:
# submit_part_1(part_1, DAY, 2022)
# submit_part_2(part_2, DAY, 2022)